In [18]:
#Importamos las librerias a utilizar.
import numpy as np
import neattext as nt
import joblib as jbl
import os

In [19]:
def resultado_class(X):
    media = np.mean(X)
    
    if media >=0.10 and media < 0.15:
        return "Es poco probable que el tweet sea de un desastre"
    elif media >=0.15 and media <0.25:
        return "Es bastante probable que el tweet sea de un desastre"
    elif media >= 0.25 and media <0.50:
        return "Es altamente probable que el tweet sea de un desastre"
    elif media >=0.50 and media <=1:
        return "El tweet es de un desastre"
    else: 
        return "El Tweet no es de un desastre"

In [20]:
def limpiar(text):

    text = nt.TextFrame(str(text)).remove_stopwords(lang = "en")
    text = nt.TextFrame(str(text)).remove_emails()
    text = nt.TextFrame(str(text)).remove_numbers()
    text = nt.TextFrame(str(text)).remove_phone_numbers()
    text = nt.TextFrame(str(text)).remove_btc_address()
    text = nt.TextFrame(str(text)).remove_urls()
    text = nt.TextFrame(str(text)).remove_emojis()
    text = nt.TextFrame(str(text)).remove_html_tags()
    text = nt.TextFrame(str(text)).remove_puncts()
    text = nt.TextFrame(str(text)).remove_special_characters()
    text = nt.TextFrame(str(text)).remove_stopwords(lang = "en")
    text = text.word_tokens()
    return text


In [21]:
count_vec= jbl.load("vectorizer")
tfidf_vec= jbl.load("tfdvectorizer")

In [22]:
model_naive = jbl.load("naive_bayes")
model_logistic = jbl.load("logistic_regresion")
model_logistic_2 = jbl.load("logistic_regresion_V")

In [91]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State

import warnings
warnings.filterwarnings("ignore")

In [95]:
app = dash.Dash()

app.layout = html.Div(style={'background-image': 'url("https://about.twitter.com/content/dam/about-twitter/en/brand-toolkit/brand-hero-desktop.jpg.twimg.1920.jpg")', }, id = 'parent', children = [
    html.H1(id = 'H1', children = 'Predicción de Tweets', style = {'textAlign':'center',\
                                            'marginTop':50,'marginBottom':50, 'font-family':'Courier New', 'font-size':'70px', 'color':'#00acee'}),
        
        html.H2("Modelos",  style ={'font-family':'Courier New', 'color':'#00acee'}),  #para texto
        dcc.Dropdown(id = 'dropdown', #menu de opciones 
        options = [
            {'label':'Naive - Bayes (Más recomendado)', 'value':'bayes'}, #lsita de opciones "label" el valor que se muestra en pantalla
            {'label': 'Regresion Logistica + Vectorizador (recomendado)', 'value':'regresionLV'}, #value es el valor del dropdown
            {'label': 'Regresion Logistica (menos recomendado)', 'value':'regresionL'},
            {'label':'Todos los Modelos', 'value':'models'},
            ],
        value = 'bayes', style = {'font-family':'Courier New'}), #valor inicial 
    
    dcc.Textarea(
        id='text_area',
        value='',
        style={'width': '99.5%', 'height': 300, 'font-family':'Courier New', 'font-size':'20px'},
    ),
    html.Button('Predecir', id='button', n_clicks=0, style ={'font-family':'Courier New', "font-size": "15px", 'width': '10%', 'height': 40, 'background-color':'#3b83bd', 'color':'white', 'border-radius': '4px'}),
    html.Button("Limpiar Texto", id = "button2", n_clicks = 0, style ={'font-family':'Courier New', "font-size": "15px", 'width': '10%', 'height': 40, 'background-color':'#006494', 'color':'white', 'border-radius': '4px'}),
    html.Div(id='resultado', style={'whiteSpace': 'pre-line', "font-size": "large", 'font-family':'Courier New'}),


    dcc.Graph(
        figure={
            'data': [
                {'x': ['Prueba 1', 'Prueba 2', 'Prueba 3', 'Prueba 4'], 'y': [0.7267, 0.8322, 0.8578, 0.8775], 'type': 'bar', 'name': 'Accuracy'},
            ],
            'layout': {
                'title': 'Accuracy Regresión Logística'
            }}),
    
    dcc.Graph(
        figure={
            'data': [
                {'x': ['Prueba 1', 'Prueba 2', 'Prueba 3', 'Prueba 4'], 'y': [0.8051470588235294, 0.7871781397792958, 0.8034682080924855, 0.7861271676300579], 'type': 'bar', 'name': 'Accuracy'},
            ],
            'layout': {
                'title': 'Accuracy Nave - Bayes'
            }}),

    dcc.Graph(
            figure={
                'data': [
                    {'x': ['Prueba 1', 'Prueba 2', 'Prueba 3', 'Prueba 4'], 'y': [0.7988445378151261, 0.792433000525486, 0.7961114030478192, 0.7819232790331057], 'type': 'bar', 'name': 'Accuracy'},
                ],
                'layout': {
                    'title': 'Accuracy Regresion Logistica + Vectorizador'
                }})

])
    
    
@app.callback(
    Output('resultado', 'children'),
    Input("dropdown", "value"),
    Input('button', 'n_clicks'),
    State('text_area', 'value')
)

def update_output(dropdown_value,n_clicks, text):
    
    if n_clicks > 0:
        clean_text = limpiar(text)
        if dropdown_value == "bayes":
            x_test=count_vec.transform(clean_text)
            resultado = model_naive.predict(x_test)
            text = ""
            
            return 'Usando NAIVE - BAYES: \n{}'.format(resultado_class(resultado))
       
        elif dropdown_value == "regresionL":
            x_test=count_vec.transform(clean_text)
            resultado = model_logistic.predict(x_test)
            text = ""
            
            return 'Usando Regresión Lineal:  \n{}'.format(resultado_class(resultado))
            
        elif dropdown_value == "regresionLV":
            x_test=tfidf_vec.transform(clean_text)
            resultado = model_logistic_2.predict(x_test)
            text = ""
            
            return 'Usando Regresión Lineal +  Vectorizador: \n{}'.format(resultado_class(resultado))
        
        else:
            x_vec = count_vec.transform(clean_text)
            x_tfidf = tfidf_vec.transform(clean_text)
            
            res_naive = model_naive.predict(x_vec)
            res_LR = model_logistic.predict(x_vec)
            res_LRV = model_logistic_2.predict(x_tfidf)
            text = ""
            
            return 'Usando NAIVE - BAYES: {}'.format(resultado_class(res_naive)) + " \n " + 'Usando Regresión Lineal: {}'.format(resultado_class(res_LR)) + " \n " + 'Usando Regresión Lineal +  Vectorizador: {}'.format(resultado_class(res_LRV)) 

    
@app.callback(Output('button','n_clicks'),
             [Input('button2','n_clicks')])

def update(reset):
    return 0



In [96]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2021 16:04:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2021 16:04:10] "POST /_dash-update-component HTTP/1.1" 200 -
